- Lees VIEW in fd_stats_resultaten. Dit geeft een tabel met resultaten met datumkolom als date én geconformeerde teamnamen.
- haal update datum op en maak deelselectie van fd_stats_resultaten (vanaf update datum)
- check of wedstrijd al voorkomt in db en voeg eventueel toe
- uniek nummer wordt automatisch gegenereerd
- daarna hetzelfde voor odds
- update de updatedatum

In [2]:
import pandas as pd
import sqlalchemy
import psycopg2

# connection variables
URL = 'football-fortunes.cj5o1qrvsyia.eu-west-1.rds.amazonaws.com'
database_name = 'postgres'
port = 5432
username = 'master'
ww = 'wachtwoord'

# Create SQL Alchemy database connection
db_url = f"postgresql://{username}:{ww}@{URL}:{port}/{database_name}"
engine = sqlalchemy.create_engine(db_url)
sqal_connection = engine.connect()

# Create Psychop connection
psy_connection = psycopg2.connect(
    host = URL,
    port = port,
    database = database_name,
    user= username,
    password = ww
)
psy_cursor = psy_connection.cursor()

# Haal alle wedstrijden op uit fd_stats_resultaten_VIEW welke nog niet in dwa.wedstrijden staan
aantal_nieuw_fd_stats = 0
wedstrijden_fd_stats = pd.read_sql("""
                                       select 
                                            fsr.div
                                            ,fsr.hometeam
                                            ,fsr.awayteam
                                        from dsa.fd_stats_resultaten_VIEW fsr 
                                        left outer join dwa.d_wedstrijden dw 
                                            on fsr.hometeam = dw.hometeam and fsr.awayteam = dw.awayteam 
                                        where s_wedstrijd is null
                                        group by fsr.div, fsr.hometeam, fsr.awayteam
                                       ;""", sqal_connection)

for i, wedstrijd in wedstrijden_fd_stats.iterrows():
    psy_cursor.execute("""
                        INSERT INTO dwa.d_wedstrijden (competitie, hometeam, awayteam)
                        VALUES (%s, %s, %s);                        
                        """
                        ,(wedstrijd[0], wedstrijd[1], wedstrijd[2]))  
    aantal_nieuw_fd_stats += 1 
print(aantal_nieuw_fd_stats)

# Haal alle wedstrijden op uit oddapi_odds_VIEW welke nog niet in dwa.wedstrijden staan
aantal_nieuw_oddapi_odds = 0
wedstrijden_oddapi_odds = pd.read_sql("""select
                                            oo.div
                                            ,oo.hometeam
                                            ,oo.awayteam
                                        from dsa.oddapi_odds_VIEW oo 
                                        left outer join dwa.d_wedstrijden dw 
                                            on oo.hometeam = dw.hometeam and oo.awayteam = dw.awayteam 
                                        where s_wedstrijd is null
                                        group by oo.div, oo.hometeam, oo.awayteam
                                       ;""", sqal_connection)

for i, wedstrijd in wedstrijden_oddapi_odds.iterrows():
    psy_cursor.execute("""
                        INSERT INTO dwa.d_wedstrijden (competitie, hometeam, awayteam)
                        VALUES (%s, %s, %s);                        
                        """
                        ,(wedstrijd[0], wedstrijd[1], wedstrijd[2]))  
    aantal_nieuw_oddapi_odds += 1 
print(aantal_nieuw_oddapi_odds)

# update upate_tabel
psy_cursor.execute("""
                    UPDATE dsa.update
                    SET update_datum = current_date
                    WHERE tabelnaam = 'd_wedstrijden';                      
                    """)

# Commit de transactie en sluit de verbinding
sqal_connection.commit()
psy_connection.commit()
psy_cursor.close()
sqal_connection.close()
psy_connection.close() 

9141
14
